In [4]:
import pymongo
import numpy as np
from nltk.corpus import cmudict
cmudict = cmudict.dict()
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()
import time
import stylometry
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'stylometry'

In [2]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'gutenberg_db'), 'gutenberg_collection')

In [1]:
text = collection.find({'file': 'project_gutenberg/2701.txt'}, {'text': 1})[0]['text']
text

NameError: name 'collection' is not defined

In [ ]:
s = time.time()
doc = nlp(text)
tokens = [token.orth_.lower() for token in doc if not token.is_punct and token if not token.is_stop]
sentences = [sent.string.strip() for sent in doc.sents]
Print(f'Took {time.time() - s}s to process')